In [5]:
%pip -q install google-genai

In [1]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [2]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install google-adk


In [3]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [4]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [5]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente buscador #################################################
        instruction=""" Você é um assistente de pesquisa especializado em determinar a probabilidade de veracidade de um tópico com base na análise de notícias recentes e históricas.

Sua tarefa é usar a ferramenta de busca do Google (google_search) para recuperar notícias relacionadas ao seguinte tópico abaixo.

Ao realizar a busca e analisar os resultados, siga estas etapas:

1.  **Busque as últimas notícias relevantes sobre o tópico:** Concentre-se em notícias publicadas recentemente (idealmente nos últimos meses e de fontes seguras e/ou institucionais).
2.  **Analise a probabilidade de veracidade com base na comparação:**
    * **Concordância:** Verifique se múltiplas fontes confiáveis reportam informações semelhantes sobre o tópico. Quanto maior a concordância entre fontes reputáveis, maior a probabilidade de veracidade.
    * **Tom:** Avalie o tom das notícias. Notícias com tom neutro e factual de fontes confiáveis tendem a indicar maior probabilidade de veracidade do que notícias com tom sensacionalista ou excessivamente opinativo.
    * **Entusiasmo/Cobertura:** Se um tópico gerar uma grande quantidade de notícias e discussões de fontes diversas e confiáveis, isso pode indicar sua relevância e potencial veracidade (embora não garanta).
3.  **Lide com poucos resultados ou falta de entusiasmo:**
    * Se houver poucas notícias recentes ou pouco entusiasmo em torno do tópico, realize uma busca por notícias que possam ter desencadeado o tema, buscando informações de até dois anos antes.
    * Avalie os fatos apresentados nessas notícias mais antigas para entender a origem e o desenvolvimento do tema.
4.  **Formate cada notícia encontrada:** Para cada notícia relevante, inclua no seu relatório:
    * **Data de Publicação:** (Extraída da notícia)
    * **Autor/Fonte:** (Nome do veículo ou autor, se disponível)
    * **Título da Notícia:**
    * **Breve Resumo/Trecho Relevante:** (Um ou dois parágrafos destacando as informações chave)

Seu relatório deve ser um resumo das notícias encontradas e sua avaliação inicial da probabilidade de veracidade do tópico com base na sua análise comparativa, tom e cobertura.
Mencione se houve poucos resultados recentes e como você procedeu para investigar a origem do tema.
        """,
        description="Agente que busca notícias, imagens e vídeos no google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico:{topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    noticias_buscadas = call_agent(buscador, entrada_do_agente_buscador)
    return noticias_buscadas


In [9]:
################################################
# --- Agente 2: Avaliador de noticias --- #
################################################
def agente_avaliador(topico, noticias_buscadas):
    avaliador = Agent(
        name="agente_avaliador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um assistente de análise de informações, responsável por avaliar o relatório produzido pelo `agente_buscador` sobre a veracidade de um tópico.

Sua tarefa é coletar o relatório do `agente_buscador` e realizar uma análise aprofundada para fornecer informações valiosas para o `agente_redator`.

Com base no relatório do `agente_buscador`, realize as seguintes ações:

1.  **Classificação Hierárquica da Veracidade:** Classifique a probabilidade de veracidade da informação em uma escala (por exemplo: Alta Probabilidade,
      Média Probabilidade, Baixa Probabilidade, Improvável, Não Conclusivo). Justifique sua classificação com base na concordância, tom e entusiasmo observados nas notícias.
2.  **Análise Qualitativa da Veracidade:** Descreva qualitativamente o nível de suporte factual encontrado. Existem evidências concretas e verificáveis?
      As fontes são confiáveis e consistentes? Há contradições ou informações conflitantes?
3.  **Análise Quantitativa da Veracidade:** Se possível, quantifique a força do suporte (por exemplo: "8 em cada 10 fontes principais reportam informações consistentes").
4.  **Identificação de Viés Ideológico:** Analise as fontes das notícias em busca de possíveis vieses ideológicos que possam influenciar a forma como a informação é apresentada.
      Mencione os possíveis vieses identificados e como eles podem afetar a interpretação da informação.
5.  **Contextualização da Informação:** Com base nas notícias e em seu conhecimento geral, forneça o contexto provável em que a informação está inserida.
      Quais eventos ou situações podem estar relacionados a essa informação?
6.  **Conteúdo de Apoio para o Agente Redator:** Forneça uma lista concisa de pontos chave, fatos confirmados (se houver),
     evidências claras e o contexto identificado que o `agente_redator` pode usar para escrever seu relatório. Priorize informações com forte suporte factual.

Seu relatório deve ser uma análise estruturada da probabilidade de veracidade, viés ideológico, contexto e um resumo de conteúdo chave para o `agente_redator`.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_avaliador = f"Tópico:{topico}\nLançamentos buscados: {noticias_buscadas}"
    # Executa o agente
    avaliacao_critica = call_agent(avaliador, entrada_do_agente_avaliador)
    return avaliacao_critica

In [10]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, avaliacao_critica):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
Você é um redator de relatórios informativos, responsável por apresentar um parecer claro e direto sobre a veracidade de uma informação com base na análise fornecida pelo `agente_avaliador`.

Sua tarefa é escrever um relatório conciso que informe o leitor sobre a probabilidade de veracidade da informação original.

Com base no relatório do `agente_avaliador`, redija um relatório que inclua os seguintes elementos:

1.  **Parecer sobre a Veracidade:** Apresente de forma clara o parecer sobre a probabilidade de veracidade da informação (usando a classificação do `agente_avaliador`).
2.  **Veículo de Disseminação (se conhecido):** Mencione qual foi o veículo de informação que disseminou a notícia original (se essa informação estiver disponível no input ou nos relatórios anteriores).
3.  **Contexto da Informação:** Apresente o contexto em que a informação provavelmente está inserida, conforme identificado pelo `agente_avaliador`.
4.  **Informações Esclarecedoras:** Forneça informações chave que ajudem o leitor a entender a mensagem com maior facilidade. Baseie-se nos fatos confirmados e nas evidências claras identificadas pelo `agente_avaliador`.
5.  **Fontes Principais (opcional):** Se relevante e conciso, você pode mencionar as principais fontes que sustentam o parecer (sem entrar em detalhes exaustivos).

Seu relatório deve ser um texto claro, direto e informativo, resumindo a análise de veracidade, o contexto e as informações essenciais para o entendimento do leitor. Evite jargões e seja objetivo.
            """,
        description="Agente redator de posts esclarecedores para publico geral"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {avaliacao_critica}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [11]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um revisor de textos com foco em clareza e precisão. Sua tarefa é revisar o relatório escrito pelo `agente_redator` em busca de frases ambíguas que possam levar a interpretações equivocadas.

Ao revisar o relatório do `agente_redator`, siga estas etapas:

1.  **Leia atentamente o relatório.**
2.  **Identifique frases que possam ter múltiplos significados ou que não sejam claras o suficiente.** Procure por pronomes com referentes obscuros, construções gramaticais complexas que dificultam a compreensão,
      termos vagos ou imprecisos, e qualquer outra formulação que possa gerar confusão no leitor.
3.  **Sugira revisões para tornar as frases mais claras e diretas.** Seu objetivo é eliminar qualquer ambiguidade e garantir que a mensagem seja transmitida de forma inequívoca.
4.  **Mantenha o tom e o estilo do relatório original o máximo possível**, focando apenas na eliminação da ambiguidade.

Seu relatório deve ser uma lista das frases ambíguas encontradas e suas sugestões de revisão para maior clareza. Se não encontrar ambiguidades, informe isso explicitamente.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [14]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Verificação de notícias 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("Por favor, digite o TÓPICO sobre o qual você quer verificar a informação ")

# Inserir lógica do sistema de agentes ################################################

if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print("Maravilha! Vamos trabalhar nesse topico")

  noticias_buscadas = agente_buscador(topico,data_de_hoje)
  avaliacao_critica = agente_avaliador(topico,noticias_buscadas)
  redacao = agente_redator(topico,avaliacao_critica)
  print("\n--- Rascunho Gerado ---")
  display(to_markdown(redacao))
  print("--------------------------------")

  redacao_revisada = agente_revisor(topico,redacao)
  print("\n--- Rascunho Revisado ---")
  display(to_markdown(redacao_revisada))
  print("--------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: É verdade que a Bela Campos brigou com o Caua Reymond nos bastidores da novela?
Maravilha! Vamos trabalhar nesse topico e criar os posts

--- Rascunho Gerado ---


> ## Relatório sobre a Veracidade da Informação: Possível Conflito entre Bela Campos e Cauã Reymond nos Bastidores de "Vale Tudo"
> 
> **Parecer sobre a Veracidade:** Alta probabilidade de veracidade. Múltiplas fontes de notícias relatam um conflito entre os atores.
> 
> **Veículo de Disseminação:** A informação foi disseminada por diversos veículos de notícias online, incluindo Terra, CNN Brasil, Omelete, Revista Raça Brasil, Portal Em Tempo, NaTelinha e Veja.
> 
> **Contexto da Informação:** A notícia está inserida no contexto da produção da novela "Vale Tudo". O conflito teria surgido devido a tensões nos bastidores, possivelmente relacionadas a diferenças de opinião sobre a atuação ou o ritmo de trabalho.
> 
> **Informações Esclarecedoras:**
> 
> *   Várias fontes noticiaram que Bela Campos teria feito reclamações sobre o comportamento de Cauã Reymond durante as gravações.
> *   A direção da Globo teria intervindo para mediar a situação e evitar maiores conflitos.
> *   Cauã Reymond se pronunciou sobre a polêmica, minimizando a ocorrência de uma briga.
> *   Alice Wegmann e Mariana Goldfarb manifestaram apoio a Bella Campos.
> 
> Apesar da negativa de Cauã Reymond, a quantidade de fontes que reportam o incidente e os detalhes consistentes sugerem que houve, de fato, tensões e desentendimentos nos bastidores da novela "Vale Tudo".
> 


--------------------------------

--- Rascunho Revisado ---


> ## Análise de Ambiguidade e Sugestões de Revisão
> 
> Após revisar o relatório, identifiquei as seguintes frases que podem gerar alguma ambiguidade e sugiro revisões para aumentar a clareza:
> 
> 1.  **Frase:** "A direção da Globo teria intervindo para mediar a situação e evitar maiores conflitos."
>     *   **Ambiguidade:** Não está claro *qual* situação a direção da Globo mediou.
>     *   **Sugestão de Revisão:** "A direção da Globo teria intervindo para mediar o conflito entre Bela Campos e Cauã Reymond e evitar maiores tensões."
> 
> 2.  **Frase:** "Cauã Reymond se pronunciou sobre a polêmica, minimizando a ocorrência de uma briga."
>     *   **Ambiguidade:** O que significa "minimizando a ocorrência de uma briga"? Ele negou completamente, ou admitiu algo, mas de menor importância?
>     *   **Sugestão de Revisão:** "Cauã Reymond se pronunciou sobre a polêmica, negando veementemente a ocorrência de uma briga" ou "Cauã Reymond se pronunciou sobre a polêmica, admitindo desentendimentos, mas minimizando-os como uma briga." (A escolha depende da declaração real dele)
> 
> 3.  **Frase:** "Alice Wegmann e Mariana Goldfarb manifestaram apoio a Bella Campos."
>     *   **Ambiguidade:** Como elas manifestaram apoio? Publicamente? Em que contexto?
>     *   **Sugestão de Revisão:** "Alice Wegmann e Mariana Goldfarb manifestaram apoio a Bella Campos publicamente através de suas redes sociais" ou "Alice Wegmann e Mariana Goldfarb manifestaram apoio a Bella Campos nos bastidores da novela". (A escolha depende de como o apoio foi manifestado)
> 
> As demais frases do relatório parecem claras e diretas.


--------------------------------
